<a href="https://colab.research.google.com/github/gitusermat/Deep-Learning/blob/main/NLP_Mathias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install scikit-learn
!pip install pandas
!pip install numpy
!pip install matplotlib

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset (assuming it's in CSV format)
data = pd.read_csv("eng.csv")

# Let's inspect the data
print(data.head())

# Preprocessing: Lowercase the text
data['text'] = data['text'].str.lower()

# Splitting into training and validation sets (80/20 split)
train_data, val_data = train_test_split(data, test_size=0.2, random_state=42)

# Separate features and labels
X_train = train_data['text'].tolist()
y_train = train_data[['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']].values

X_val = val_data['text'].tolist()
y_val = val_data[['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']].values

print(f"Training Samples: {len(X_train)}, Validation Samples: {len(X_val)}")


                        id                                               text  \
0  eng_train_track_a_00001                                But not very happy.   
1  eng_train_track_a_00002  Well she's not gon na last the whole song like...   
2  eng_train_track_a_00003  She sat at her Papa's recliner sofa only to mo...   
3  eng_train_track_a_00004                    Yes, the Oklahoma city bombing.   
4  eng_train_track_a_00005                       They were dancing to Bolero.   

   Anger  Fear  Joy  Sadness  Surprise  
0      0     0    1        1         0  
1      0     0    1        0         0  
2      0     0    0        0         0  
3      1     1    0        1         1  
4      0     0    1        0         0  
Training Samples: 2214, Validation Samples: 554


In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel
import torch

# Load the pre-trained XLM-R tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
xlm_roberta_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')

# Function to tokenize and encode text for XLM-R
def tokenize_and_encode(text_list):
    encoding = tokenizer(
        text_list,
        max_length=128,
        truncation=True,
        padding=True,
        return_tensors='pt'
    )
    return encoding

# Tokenize and encode the training and validation text
X_train_encoded = tokenize_and_encode(X_train)
X_val_encoded = tokenize_and_encode(X_val)

# Example to show encoded text shapes
print("Training Encoded Shape:", X_train_encoded['input_ids'].shape)
print("Validation Encoded Shape:", X_val_encoded['input_ids'].shape)

Training Encoded Shape: torch.Size([2214, 107])
Validation Encoded Shape: torch.Size([554, 117])


In [ ]:
import torch.nn as nn
from transformers import XLMRobertaModel

# Define the model using XLM-RoBERTa
class EmotionClassifier(nn.Module):
    def __init__(self, xlm_roberta_model, num_labels):
        super(EmotionClassifier, self).__init__()
        self.xlm_roberta = xlm_roberta_model
        self.dropout = nn.Dropout(0.3)
        self.classifier = nn.Linear(xlm_roberta_model.config.hidden_size, num_labels)

    def forward(self, input_ids, attention_mask):
        outputs = self.xlm_roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_state = outputs.last_hidden_state
        cls_output = hidden_state[:, 0, :]  # Use [CLS] token for classification
        cls_output = self.dropout(cls_output)
        logits = self.classifier(cls_output)
        return logits

# Load XLM-RoBERTa and initialize the model
num_labels = 5  # Number of emotion classes: Anger, Fear, Joy, Sadness, Surprise
model = EmotionClassifier(xlm_roberta_model, num_labels)

# Check if GPU is available and move the model to GPU if possible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


EmotionClassifier(
  (xlm_roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True

In [ ]:
from torch.optim import AdamW

# Optimizer
# Use a slightly increased learning rate to speed up convergence and regularization to prevent overfitting.
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)


In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Debugging: Print sizes of input and labels before processing
print("Before Alignment:")
print("Input IDs shape:", X_train_encoded['input_ids'].shape)
print("Attention Mask shape:", X_train_encoded['attention_mask'].shape)
print("Labels shape:", len(y_train))

# Ensure the labels match the number of tokenized inputs
min_size = min(X_train_encoded['input_ids'].shape[0], len(y_train))
X_train_encoded['input_ids'] = X_train_encoded['input_ids'][:min_size]
X_train_encoded['attention_mask'] = X_train_encoded['attention_mask'][:min_size]
y_train = y_train[:min_size]

min_size = min(X_val_encoded['input_ids'].shape[0], len(y_val))
X_val_encoded['input_ids'] = X_val_encoded['input_ids'][:min_size]
X_val_encoded['attention_mask'] = X_val_encoded['attention_mask'][:min_size]
y_val = y_val[:min_size]

# Convert labels to tensors with appropriate shape
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)  # Shape: (num_samples, num_labels)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)      # Shape: (num_samples, num_labels)

# Verify shapes after adjustment
print("After Alignment:")
print("Input IDs shape:", X_train_encoded['input_ids'].shape)
print("Attention Mask shape:", X_train_encoded['attention_mask'].shape)
print("Labels shape:", y_train_tensor.shape)

# Create TensorDatasets
train_dataset = TensorDataset(
    X_train_encoded['input_ids'],
    X_train_encoded['attention_mask'],
    y_train_tensor
)
val_dataset = TensorDataset(
    X_val_encoded['input_ids'],
    X_val_encoded['attention_mask'],
    y_val_tensor
)

# Create DataLoaders
# Reduced batch size to ensure stability when training on multilingual data
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)  # Adjust batch size as per GPU memory
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Verify DataLoader functionality
for batch in train_loader:
    input_ids, attention_mask, labels = batch
    print("Batch Input IDs Shape:", input_ids.shape)
    print("Batch Attention Mask Shape:", attention_mask.shape)
    print("Batch Labels Shape:", labels.shape)
    break

Before Alignment:
Input IDs shape: torch.Size([2214, 107])
Attention Mask shape: torch.Size([2214, 107])
Labels shape: 2214
After Alignment:
Input IDs shape: torch.Size([2214, 107])
Attention Mask shape: torch.Size([2214, 107])
Labels shape: torch.Size([2214, 5])
Batch Input IDs Shape: torch.Size([8, 107])
Batch Attention Mask Shape: torch.Size([8, 107])
Batch Labels Shape: torch.Size([8, 5])


In [ ]:
import torch.nn.utils
from torch.optim.lr_scheduler import OneCycleLR
from torch.cuda.amp import GradScaler, autocast
import time

# Mixed precision training setup (Updated deprecated usage)
scaler = torch.amp.GradScaler()

# Define the training loop with improvements
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler=None, epochs=5, patience=2):
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    wait = 0

    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        start_time = time.time()

        # Training phase
        for batch_idx, batch in enumerate(train_loader):
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            # Use autocast for mixed precision training
            with torch.amp.autocast(device_type='cuda'):
                logits = model(input_ids=input_ids, attention_mask=attention_mask)
                loss = criterion(logits, labels)

            # Backward pass and optimization using gradient scaling
            scaler.scale(loss).backward()

            # Clip the gradients to avoid exploding gradients
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            # Perform optimization
            scaler.step(optimizer)
            scaler.update()

            # Accumulate the loss to calculate the average later
            total_loss += loss.item()

            # Print every 50 batches to track progress
            if (batch_idx + 1) % 50 == 0:
                print(f"Epoch [{epoch + 1}/{epochs}], Step [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

        # Calculate average training loss for the epoch
        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"Epoch {epoch + 1}, Average Training Loss: {avg_train_loss:.4f}, Time taken: {time.time() - start_time:.2f}s")

        # Validation phase
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                with torch.amp.autocast(device_type='cuda'):
                    logits = model(input_ids=input_ids, attention_mask=attention_mask)
                    val_loss = criterion(logits, labels)
                    total_val_loss += val_loss.item()

        # Calculate average validation loss for the epoch
        avg_val_loss = total_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Epoch {epoch + 1}, Validation Loss: {avg_val_loss:.4f}")

        # Scheduler step (if used)
        if scheduler:
            scheduler.step()

        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            wait = 0
            print(f"Validation loss improved to {best_val_loss:.4f}, resetting patience.")
        else:
            wait += 1
            print(f"No improvement in validation loss. Patience: {wait}/{patience}")
            if wait >= patience:
                print("Early stopping triggered.")
                break

    return train_losses, val_losses

# Scheduler - Use OneCycleLR to have more dynamic adjustments to learning rate
scheduler = OneCycleLR(optimizer, max_lr=3e-5, epochs=5, steps_per_epoch=len(train_loader))

# Criterion for multi-label classification
criterion = torch.nn.BCEWithLogitsLoss()

# Run the training loop with corrected settings
train_losses, val_losses = train_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    epochs=3,  # Reduced number of epochs to prevent overfitting
    patience=2  # Lower patience to stop training early if overfitting starts
)

/usr/local/lib/python3.10/dist-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


In [ ]:
import matplotlib.pyplot as plt

# Plotting the loss curves
def plot_loss(train_losses, val_losses):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(10, 6))
    plt.plot(epochs, train_losses, label='Training Loss', marker='o')
    plt.plot(epochs, val_losses, label='Validation Loss', marker='o')

    plt.title('Training and Validation Loss over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.show()

# Plot the losses
plot_loss(train_losses, val_losses)


NameError: name 'train_losses' is not defined

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report
import numpy as np

# Get predictions and calculate F1-score
def evaluate(model, data_loader):
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            logits = model(input_ids, attention_mask)
            preds = torch.sigmoid(logits).cpu().numpy()  # Convert to probabilities

            all_labels.append(labels.cpu().numpy())
            all_preds.append((preds > 0.5).astype(int))  # Apply threshold

    # Flatten the lists
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)

    # Calculate metrics
    f1 = f1_score(all_labels, all_preds, average='micro')
    precision = precision_score(all_labels, all_preds, average='micro')
    recall = recall_score(all_labels, all_preds, average='micro')
    print(f"Micro F1-score: {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print("\nClassification Report:\n", classification_report(all_labels, all_preds))

# Evaluate on validation data
evaluate(model, val_loader)

Micro F1-score: 0.6575
Precision: 0.6675
Recall: 0.6479

Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.25      0.31        61
           1       0.71      0.86      0.78       314
           2       0.72      0.53      0.61       134
           3       0.58      0.58      0.58       171
           4       0.68      0.56      0.61       172

   micro avg       0.67      0.65      0.66       852
   macro avg       0.62      0.56      0.58       852
weighted avg       0.66      0.65      0.65       852
 samples avg       0.61      0.60      0.58       852



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
import pandas as pd

# Define your emotion labels (ensure these are in the same order as your model's output)
emotion_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']

# Helper function to decode predictions from binary array to emotion labels
def decode_predictions(predictions):
    decoded_preds = []
    for pred in predictions:
        decoded_preds.append([emotion_labels[i] for i, val in enumerate(pred) if val == 1])
    return decoded_preds

# Helper function to decode true labels
def decode_true_labels(labels):
    decoded_labels = []
    for label in labels:
        decoded_labels.append([emotion_labels[i] for i, val in enumerate(label) if val == 1])
    return decoded_labels

# Function to visualize output
def visualize_output(model, data_loader, text_data):
    model.eval()
    all_labels = []
    all_preds = []
    input_texts = []

    with torch.no_grad():
        for batch in data_loader:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)

            # Forward pass through the model
            outputs = model(input_ids, attention_mask)
            preds = torch.sigmoid(outputs).cpu().numpy()  # Convert logits to probabilities

            all_labels.append(labels.cpu().numpy())
            all_preds.append(preds > 0.5)  # Apply threshold (0.5)

    # Flatten predictions and labels
    all_labels = np.concatenate(all_labels, axis=0)
    all_preds = np.concatenate(all_preds, axis=0)

    # Decode predictions and true labels to emotion labels
    predicted_emotions = decode_predictions(all_preds)
    true_emotions = decode_true_labels(all_labels)

    # Create a DataFrame to visualize the output
    results_df = pd.DataFrame({
        'Text': text_data,  # Original text (before tokenization)
        'Predicted Emotions': predicted_emotions,
        'True Emotions': true_emotions
    })

    return results_df

# Example usage:
# Assuming `X_val` contains the original text snippets used for validation
results_df = visualize_output(model, val_loader, X_val)

# Display the first few rows of predictions vs actual
print(results_df.head(10))  # Show first 10 samples


In [ ]:
# # Convert predictions to readable format (e.g., list of labels)
# emotion_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']

# def decode_predictions(predictions):
#     decoded_preds = []
#     for pred in predictions:
#         decoded_preds.append([emotion_labels[i] for i, val in enumerate(pred) if val == 1])
#     return decoded_preds

# # Decode predictions and true labels (if available)
# predicted_emotions = decode_predictions(all_preds)
# true_emotions = decode_predictions(y_val)

# # Combine into a DataFrame to visualize
# results_df = pd.DataFrame({'Text': X_val, 'Predicted Emotions': predicted_emotions, 'True Emotions': true_emotions})

# # Show the first few rows
# print(results_df.head())


In [ ]:
### result on test set

In [ ]:
# Assuming the new test data is a CSV
new_data = pd.read_csv("eng_a.csv")

# Inspect the new data
print(new_data.head())

# Preprocessing: Ensure text is lowercased
new_data['text'] = new_data['text'].str.lower()

# Extract the text from the new data
X_new = new_data['text'].tolist()


In [ ]:
# Tokenize and encode the new data
X_new_encoded = tokenize_and_encode(X_new)

# Example to show the shape of the tokenized new data
print(X_new_encoded['input_ids'].shape)


In [ ]:
# Set model to evaluation mode
model.eval()

# Prepare DataLoader for new data
from torch.utils.data import DataLoader, TensorDataset

new_dataset = TensorDataset(X_new_encoded['input_ids'], X_new_encoded['attention_mask'])
new_loader = DataLoader(new_dataset, batch_size=16, shuffle=False)

# Get predictions
all_new_preds = []

with torch.no_grad():
    for batch in new_loader:
        input_ids, attention_mask = batch
        input_ids = input_ids.to(device)
        attention_mask = attention_mask.to(device)

        # Forward pass to get logits
        outputs = model(input_ids, attention_mask)
        preds = torch.sigmoid(outputs).cpu().numpy()  # Convert logits to probabilities

        # Apply threshold (e.g., 0.5)
        all_new_preds.append(preds > 0.5)

# Convert list of predictions to a numpy array
all_new_preds = np.concatenate(all_new_preds, axis=0)

# Display predictions (will be a binary matrix: 1 for present, 0 for absent)
print(all_new_preds)


NameError: name 'model' is not defined

In [ ]:
# Save the trained model
torch.save(model.state_dict(), 'emotion_detection_model.pth')

# To load the model later
# model = YourModelClass()
# model.load_state_dict(torch.load('emotion_detection_model.pth'))


NameError: name 'torch' is not defined

In [ ]:
! pip install SpeechRecognition

In [ ]:
import speech_recognition as sr

# Convert speech to text
def speech_to_text():
    recognizer = sr.Recognizer()

    with sr.Microphone() as source:
        print("Please say something...")
        audio = recognizer.listen(source)

        try:
            print("Recognizing...")
            text = recognizer.recognize_google(audio)
            print(f"Text: {text}")
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None


In [ ]:
from transformers import BertModel
import torch.nn as nn

# Update model class to match the saved model's architecture
class EmotionClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EmotionClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)  # Change 'out' to 'classifier'

    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask
        )[1]
        output = self.drop(pooled_output)
        return self.classifier(output)  # Change 'out' to 'classifier'


In [ ]:
from transformers import BertTokenizer
import torch

# Update the load_model function
def load_model():
    n_classes = 5  # Number of emotions
    model = EmotionClassifier(n_classes)  # Replace with your model class
    model.load_state_dict(torch.load('emotion_detection_model.pth'))
    model.eval()
    return model


# Predict emotions from text
def predict_emotions(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = torch.sigmoid(outputs).cpu().numpy()

    # Apply threshold (0.5)
    emotion_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
    predicted_emotions = [emotion_labels[i] for i, val in enumerate(predictions[0]) if val > 0.5]

    return predicted_emotions


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

def load_model():
    # Number of emotions (adjust if needed for your specific use case)
    n_classes = 5

    # Load a pre-trained BERT model for sequence classification
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=n_classes
    )

    # Load model weights (adjust the path to where your model is saved)
    model.load_state_dict(torch.load('emotion_detection_model.pth', map_location=torch.device('cpu')))

    # Set the model to evaluation mode
    model.eval()

    return model

def main():
    # Load model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = load_model()

    # Convert speech to text (assuming this function is defined elsewhere)
    text = speech_to_text()

    if text:
        # Predict emotions
        emotions = predict_emotions(text, model, tokenizer)

        if emotions:
            print(f"Predicted emotions: {', '.join(emotions)}")
        else:
            print("No emotions detected.")
    else:
        print("No text to process.")

if __name__ == "__main__":
    main()


In [ ]:
!pip install pyaudio


In [ ]:
# Import necessary libraries
import io
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
import speech_recognition as sr
from google.colab import output
from IPython.display import display, Javascript

# Function to record audio
def record_audio():
    # JavaScript to record audio
    display(Javascript('''
    async function recordAudio() {
        const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
        const mediaRecorder = new MediaRecorder(stream);
        const audioChunks = [];

        mediaRecorder.ondataavailable = event => {
            audioChunks.push(event.data);
        };

        mediaRecorder.onstop = async () => {
            const audioBlob = new Blob(audioChunks);
            const arrayBuffer = await audioBlob.arrayBuffer();
            const base64String = btoa(String.fromCharCode(...new Uint8Array(arrayBuffer)));
            google.colab.kernel.invokeFunction('notebook.recordingComplete', [base64String], {});
        };

        mediaRecorder.start();
        await new Promise(resolve => setTimeout(resolve, 5000)); // Record for 5 seconds
        mediaRecorder.stop();
    }
    recordAudio();
    '''))

# Function to handle audio recording completion
def handle_recording_complete(base64_audio):
    # Decode the base64 audio
    audio_data = io.BytesIO(base64.b64decode(base64_audio))

    # Use SpeechRecognition to convert audio to text
    recognizer = sr.Recognizer()

    with sr.AudioFile(audio_data) as source:
        audio = recognizer.record(source)  # Read the entire audio file

        try:
            print("Recognizing...")
            text = recognizer.recognize_google(audio)
            print(f"Text: {text}")
            return text
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return None
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return None

# Define the EmotionClassifier model
class EmotionClassifier(nn.Module):
    def __init__(self, n_classes):
        super(EmotionClassifier, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.drop = nn.Dropout(p=0.3)
        self.classifier = nn.Linear(self.bert.config.hidden_size, n_classes)

    def forward(self, input_ids, attention_mask):
        pooled_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)[1]
        output = self.drop(pooled_output)
        return self.classifier(output)

# Load the model
def load_model():
    n_classes = 5  # Number of emotions
    model = EmotionClassifier(n_classes)
    model.load_state_dict(torch.load('emotion_detection_model.pth', map_location=torch.device('cpu')))
    model.eval()
    return model

# Predict emotions from text
def predict_emotions(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', padding=True, truncation=True)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask)
        predictions = torch.sigmoid(outputs).cpu().numpy()

    # Apply threshold (0.5)
    emotion_labels = ['Anger', 'Fear', 'Joy', 'Sadness', 'Surprise']
    predicted_emotions = [emotion_labels[i] for i, val in enumerate(predictions[0]) if val > 0.5]

    return predicted_emotions

def main():
    # Load model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = load_model()

    # Record audio and convert to text
    print("Please record your audio...")
    record_audio()

    # This function handles audio conversion and returns the transcribed text
    text = handle_recording_complete(audio_data)

    if text:
        # Predict emotions
        emotions = predict_emotions(text, model, tokenizer)

        if emotions:
            print(f"Predicted emotions: {', '.join(emotions)}")
        else:
            print("No emotions detected.")
    else:
        print("No text to process.")

if __name__ == "__main__":
    main()
